In [ ]:
from pathlib import Path

from climada.util.api_client import Client

EXPOSURES_DIR = Path("data", "exposures")
HAZARDS_DIR = Path("data", "hazards")

client = Client()

In [ ]:
data_type_infos= client.list_data_type_infos()
data_type_infos

In [ ]:
data_type_groups = set([data_type_info.data_type_group for data_type_info in data_type_infos])
data_type_groups

In [ ]:
data_type_exposures = [data_type_info for data_type_info in data_type_infos if data_type_info.data_type_group == 'exposures']
data_type_exposures

In [ ]:
data_type_hazards = [data_type_info for data_type_info in data_type_infos if data_type_info.data_type_group == 'hazard']
data_type_hazards

In [ ]:
dataset_infos = client.list_dataset_infos(
    properties={
        'country_name': ['Egypt', 'Thailand'],
    }
)
dataset_infos

In [28]:
data_types_set = list(set([data_type_info.data_type for data_type_info in data_type_infos if data_type_info.data_type_group == 'hazard']))
data_types_set

['river_flood',
 'tropical_cyclone',
 'relative_cropyield',
 'hail',
 'storm_europe',
 'flood',
 'earthquake',
 'wildfire']

In [ ]:
egypt_litpop_dataset_info = client.get_dataset_info(
    data_type='litpop', 
    properties={
        'country_name': 'Egypt',
        'exponents': '(1,1)'
    }
)
egypt_litpop_dataset_info

In [ ]:
thailand_litpop_dataset_info = client.get_dataset_info(
    data_type='litpop', 
    properties={
        'country_name': 'Egypt',
        'exponents': '(1,1)'
    }
)
thailand_litpop_dataset_info

In [29]:
dataset_infos = client.list_dataset_infos(
    properties={
        'data_type':'earthquake',
        'country_name': ['Egypt'],
    }
)
dataset_infos

[DatasetInfo(uuid='508648f4-c242-441f-9ed9-904431c9b92f', data_type=DataTypeShortInfo(data_type='earthquake', data_type_group='hazard'), name='earthquake_hist_above4_818', version='v1.1', status='active', properties={'res_arcsec': '150', 'event_type': 'observed', 'spatial_coverage': 'country', 'country_iso3alpha': 'EGY', 'country_name': 'Egypt', 'country_iso3num': '818'}, files=[FileInfo(uuid='508648f4-c242-441f-9ed9-904431c9b92f', url='https://data.iac.ethz.ch/climada/40e86fb1-873b-45fb-91a8-04f1d97e76b2/earthquake_hist_above4_818.hdf5', file_name='earthquake_hist_above4_818.hdf5', file_format='hdf5', file_size=5365381, check_sum='md5:863bcd1e85990670c785f4859c3708a5')], doi=None, description='Global earthquake hazard set at 4km resolution, \nbased on USGS epicentres database and a \nsimple probabilistic sampling starting from the historic earthquake catalog,\nwith 9 synthetic events per historic record.', license='Attribution 4.0 International (CC-BY-4.0)', activation_date='2023-01-3

In [ ]:
[DatasetInfo(uuid='508648f4-c242-441f-9ed9-904431c9b92f', data_type=DataTypeShortInfo(data_type='earthquake', data_type_group='hazard'), name='earthquake_hist_above4_818', version='v1.1', status='active', properties={'res_arcsec': '150', 'event_type': 'observed', 'spatial_coverage': 'country', 'country_iso3alpha': 'EGY', 'country_name': 'Egypt', 'country_iso3num': '818'}, files=[FileInfo(uuid='508648f4-c242-441f-9ed9-904431c9b92f', url='https://data.iac.ethz.ch/climada/40e86fb1-873b-45fb-91a8-04f1d97e76b2/earthquake_hist_above4_818.hdf5', file_name='earthquake_hist_above4_818.hdf5', file_format='hdf5', file_size=5365381, check_sum='md5:863bcd1e85990670c785f4859c3708a5')], doi=None, description='Global earthquake hazard set at 4km resolution, \nbased on USGS epicentres database and a \nsimple probabilistic sampling starting from the historic earthquake catalog,\nwith 9 synthetic events per historic record.', license='Attribution 4.0 International (CC-BY-4.0)', activation_date='2023-01-30 16:08:10.151568+00:00', expiration_date=None)]

In [ ]:
dataset_infos = client.list_dataset_infos(
    properties={
        'data_type':'river_flood',
        'country_name': ['Egypt'],
        'climate_scenario': 'historical'
    }
)
dataset_infos

In [ ]:
exposure = client.get_litpop(
    country='Egypt',
    exponents=(1,1),
    dump_dir=EXPOSURES_DIR
)
exposure

In [ ]:
exposure.gdf

In [ ]:
exposure.plot_raster()

In [ ]:
from rasterio.warp import Resampling
from climada.entity import LitPop, ImpactFuncSet, ImpactFunc
from climada.hazard import Hazard
from climada.engine import Impact
from climada.util.constants import HAZ_DEMO_FL

haz_ras = Hazard.from_raster([HAZ_DEMO_FL], haz_type='FL', dst_crs=exposure.meta['crs'], transform=exposure.meta['transform'],
                             width=exposure.meta['width'], height=exposure.meta['height'],
                             resampling=Resampling.nearest)
haz_ras.intensity[haz_ras.intensity==-9999] = 0 # correct no data values
haz_ras.plot_intensity(0)

In [ ]:
import rasterio
import numpy as np
import pandas as pd

with rasterio.open('SC22000_VE__M1.grd') as src:
    array = src.read(1)

# Replace 'nodata' values with NaN if needed
array[array == src.nodata] = np.nan

# Convert to DataFrame
df = pd.DataFrame(array)
df

In [ ]:
hazard = client.get_hazard(
    hazard_type='river_flood',
    properties={
        'country_name': 'Egypt',
        'climate_scenario': 'historical'
    },
    dump_dir=HAZARDS_DIR,
)

In [ ]:
# Calculate impact

# hazard.intensity_thres = 1
impact = 